In [50]:
import re
from langchain.docstore.document import Document
#setup model
#setup
import os
import re
import fitz
import copy
import pathlib
import pandas as pd
from tqdm import tqdm
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.extensions.langchain import (
    WatsonxLLM,
)
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema.embeddings import Embeddings
from typing import List
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

load_dotenv()

#read property chat inquiry - converted to text 
with open("../data/faq/property_inquiry_for_chatbot 1.txt", "r") as f:
    data_inquiry = f.read()
docs_multi_prop = [i for i in re.split("Images:", data_inquiry)]

In [61]:
from langchain.output_parsers.json import SimpleJsonOutputParser

model = Model(
model_id=ModelTypes.LLAMA_2_70B_CHAT,
credentials={
    "apikey": os.getenv("IBM_API_KEY"),
    "url": "https://us-south.ml.cloud.ibm.com",
},
params={
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MAX_NEW_TOKENS: 1200,
    GenParams.TEMPERATURE: 0,
    GenParams.RANDOM_SEED: 12345,
},
project_id=os.getenv("PROJECT_ID"),
)
llm = WatsonxLLM(model=model)
parser = SimpleJsonOutputParser()

json_outputs = []
test = []
for e, i in tqdm(enumerate(docs_multi_prop)):
    prompt = f"""
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Convert the following text into JSON format, do not provide any explanation:

Input:
{i.strip()}

Output:
[/INST]
"""
    # print(i)

    gen = llm(prompt)
    if e == 11:
        gen = gen.replace('{\n"3 Bedrooms",\n"2 Bathrooms",\n"Parking Lot"\n}', '[\n"3 Bedrooms",\n"2 Bathrooms",\n"Parking Lot"\n]')
    if e == 14:
        gen = gen.replace('[\n{\n"Bedrooms": 4,\n"Bathrooms": 3,\n"Parking Lots": 3\n}', '{\n"Bedrooms": 4,\n"Bathrooms": 3,\n"Parking Lots": 3\n}')

    # print(gen)
    test.append(gen)
    gen2 = parser.invoke(gen)
    json_outputs.append(gen2)
    print(gen2)

    if gen2 is None:
        print(e)

1it [00:14, 14.66s/it]

{'Project': 'The MINH', 'Website Project name': 'The Minh', 'FloorPlan': [{'Bedrooms': 3, 'Bathrooms': 3, 'Parking Lots': 2}], 'Type': 'Type A', 'Size': 1607, 'Min Price': 'NA', 'Max Price': 'NA', 'Min Built-Up': 1607, 'Max Built-Up': 3010, 'Tenure': 'Freehold', 'Property Type': 'Condominium', 'Region': 'Central', 'Postcode': 50480, 'Location': 'Mont Kiara', 'State': 'Kuala Lumpur', 'Amenities': 'NA', 'Features': ['Resort style condominium with indochine influences', 'Beautifully designed swimming pool with water cascade', 'Private lift', 'Indoor scenic spa hut', 'EV charger ready via power point', 'Automated waste collection system', 'Multi-generational playground', 'Express ramp to multi-level parking', 'Certified by GreenRE']}


2it [00:26, 13.18s/it]

{'Project': 'The MINH', 'Website Project name': 'The Minh', 'FloorPlan': [{'Bedrooms': 3, 'Bathrooms': 3, 'Parking Lots': 2}], 'Type': 'Type A', 'Size': 1607, 'Min Price': 'NA', 'Max Price': 'NA', 'Min Built-Up': 1607, 'Max Built-Up': 3010, 'Tenure': 'Freehold', 'Property Type': 'Condominium', 'Region': 'Central', 'Postcode': 50480, 'Location': 'Mont Kiara', 'State': 'Kuala Lumpur', 'Amenities': 'NA', 'Features': ['Resort style condominium with indochine influences', 'Beautifully designed swimming pool with water cascade', 'Private lift', 'Indoor scenic spa hut', 'EV charger ready via power point', 'Automated waste collection system', 'Multi-generational playground', 'Express ramp to multi-level parking', 'Certified by GreenRE']}


3it [00:43, 14.93s/it]

...

{'Project': 'Residensi ZIG', 'Website Project name': 'Residensi ZIG', 'FloorPlan': {'Bedrooms': 4, 'Bathrooms': 2, 'Parking Lot': True}, 'Type': 'Type D', 'Size': 1135, 'Min Price': 340888, 'Max Price': 874888, 'Min Built-Up': 452, 'Max Built-Up': 1195, 'Tenure': 'Leasehold', 'Property Type': 'Apartment', 'Region': 'Central', 'Postcode': '52100', 'Location': 'Kepong', 'State': 'Kuala Lumpur', 'Amenities': ['Kepong Metropolitan Park 0.4KM', 'SMK Jinjang 1.1KM', 'MRT Kepong Baru Station 2.4KM', 'AEON Big Kepong 2.8KM', 'Selayang Hospital 4.6KM', 'Batu Caves 7.2Km', 'Mercure Hotel 7.2KM'], 'Features': ['Move-in Ready ID Package', 'Bike-friendly Project', 'Adaptive And Compact Living', 'Multi-generation Interaction', 'Wellness Lifestyle']}


15it [04:03, 17.11s/it]

None
14


16it [04:06, 15.40s/it]

{'name': 'John', 'age': 30, 'city': 'New York'}


In [31]:
del json_outputs[7]
# json_outputs = [i for i in json_outputs if i is not None]
df = pd.DataFrame.from_dict(json_outputs)
df = df.rename(columns={"name":"Project Name"})
df.to_excel("../data/faq/chatbot_json.xlsx")

In [33]:
json_outputs

[{'Project': 'The MINH',
  'Website Project name': 'The Minh',
  'FloorPlan': [{'Bedrooms': 3, 'Bathrooms': 3, 'Parking Lots': 2}],
  'Type': 'Type A',
  'Size': 1607,
  'Min Price': 'NA',
  'Max Price': 'NA',
  'Min Built-Up': 1607,
  'Max Built-Up': 3010,
  'Tenure': 'Freehold',
  'Property Type': 'Condominium',
  'Region': 'Central',
  'Postcode': 50480,
  'Location': "Mont' Kiara",
  'State': 'Kuala Lumpur',
  'Amenities': 'NA',
  'Features': ['Resort style condominium with indochine influences',
   'Beautifully designed swimming pool with water cascade',
   'Private lift',
   'Indoor scenic spa hut',
   'EV charger ready via power point',
   'Automated waste collection system',
   'Multi-generational playground',
   'Express ramp to multi-level parking',
   'Certified by GreenRE']},
 {'Project': 'The MINH',
  'Website Project name': 'The Minh',
  'FloorPlan': [{'Bedrooms': 3, 'Bathrooms': 3, 'Parking Lots': 2}],
  'Type': 'Type A',
  'Size': 1607,
  'Min Price': 'NA',
  'Max Price

In [65]:
print(test[14])

{
"Project": "Residensi ZIG",
"Website Project name": "Residensi ZIG",
"FloorPlan": {
"Bedrooms": 4,
"Bathrooms": 3,
"Parking Lots": 3
},
{
"Type": "Type E",
"Size": 1198,
"Min Price": 340888,
"Max Price": 874888,
"Min Built-Up": 452,
"Max Built-Up": 1195,
"Tenure": "Leasehold",
"Property Type": "Apartment",
"Region": "Central",
"Postcode": 52100,
"Location": "Kepong",
"State": "Kuala Lumpur"
},
{
"Amenities": [
"Kepong Metropolitan Park 0.4KM",
"SMK Jinjang 1.1KM",
"MRT Kepong Baru Station 2.4KM",
"AEON Big Kepong 2.8KM",
"Selayang Hospital 4.6KM",
"Batu Caves 7.2Km",
"Mercure Hotel 7.2KM"
]
"Features": [
"Move-in Ready ID Package",
"Bike-friendly Project",
"Adaptive And Compact Living",
"Multi-generation Interaction",
"Wellness Lifestyle"
]
}
]
}
